In [ ]:
# Import additional required libraries
import time
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import load_model

# Save the model
loaded_model = load_model('action_rec.h5')

# Function to preprocess the frame
def preprocess_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, (224, 224))
    return frame

# Function to display live video feed results
def live_video_feed(video_path, model, time_steps, label_encoder, live=False):
    if live:
        cap = cv2.VideoCapture(int(video_path))
    else:
        cap = cv2.VideoCapture(video_path)
        
    input_frames = []
    frame_count = 0
    
    foul_count = 0
    not_foul_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_skip == 0:
            preprocessed_frame = preprocess_frame(frame)
            input_frames.append(preprocessed_frame)

            if len(input_frames) >= time_steps:
                input_sequence = np.array(input_frames[-time_steps:]).reshape(1, time_steps, 224, 224, 1) / 255.0
                prediction = model.predict(input_sequence)
                predicted_class = label_encoder.inverse_transform([np.argmax(prediction)])[0]

                clear_output(wait=True)
                print("Predicted class:", predicted_class)
                
                if predicted_class == "Foul":
                    foul_count += 1
                else:
                    not_foul_count += 1

        frame_count += 1

    cap.release()
    
    if foul_count > not_foul_count:
        print("\nOverall classification: Foul")
    else:
        print("\nOverall classification: Not_Foul")


# Function to prompt user for input type and camera source
def get_user_input():
    input_type = input("Enter 'live' for live feed or 'video' for pre-recorded video: ").lower().strip()
    if input_type == 'live':
        camera_source = int(input("Enter the camera source (0 for default, 1 for external, etc.): "))
        return camera_source, input_type
    elif input_type == 'video':
        video_path = input("Enter the path to the pre-recorded video: ")
        return video_path, input_type
    else:
        print("Invalid input. Please try again.")
        return get_user_input()


# Get user input and display live video feed results
user_input, input_type = get_user_input()
live = input_type == 'live'
live_video_feed(user_input, model, time_steps, label_encoder, live=live)

# Function to evaluate the model and visualize the evaluation metrics
def evaluate_model(X_eval, y_eval, model, label_encoder):
    X_eval = X_eval.reshape(-1, time_steps, 224, 224, 1) / 255.0
    y_pred = np.argmax(model.predict(X_eval), axis=1)
    y_eval = np.argmax(y_eval, axis=1)

    conf_mat = confusion_matrix(y_eval, y_pred)
    target_names = [str(cls) for cls in label_encoder.classes_]
    class_report = classification_report(y_eval, y_pred, target_names=target_names)
    acc_score = accuracy_score(y_eval, y_pred)

    print("Confusion Matrix:\n", conf_mat)
    print("\nClassification Report:\n", class_report)
    print("\nAccuracy Score:", acc_score)

    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()


# Run the evaluation function to visualize the evaluation metrics
evaluate_model(X_test, y_test, model, label_encoder)
